In [44]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset
from torch.utils.data import DataLoader
import torch
import tqdm

In [45]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [46]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=80)

In [47]:
df = pd.read_csv('data/train.tsv', sep='\t', header=None)
df = df.drop(df.columns[2], axis=1)

newrows = []
for i in range(0, df.shape[0]): #Adds new rows for duplicates
    if "," in df[1].iloc[i]:
        # currEmotion = df[1].iloc[i].split(",")
        # currText = df[0].iloc[i]
        # for j in currEmotion:
        #     newrows.append([currText, j])
        pass
    else:
        newrows.append([df[0].iloc[i], int(df[1].iloc[i])])

df = pd.DataFrame(newrows)
        
train_dataset = Dataset.from_pandas(pd.DataFrame({'text': df[0], 'emotion': df[1]}))
train_dataset = train_dataset.map(tokenize_function, batched=True)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'emotion'])
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)

Map: 100%|██████████| 36308/36308 [00:12<00:00, 2900.21 examples/s]


In [48]:
df = pd.read_csv('data/dev.tsv', sep='\t', header=None)
df = df.drop(df.columns[2], axis=1)

newrows = []
for i in range(0, df.shape[0]): #Adds new rows for duplicates
    if "," in df[1].iloc[i]:
        # currEmotion = df[1].iloc[i].split(",")
        # currText = df[0].iloc[i]
        # for j in currEmotion:
        #     newrows.append([currText, j])
        pass
    else:
        newrows.append([df[0].iloc[i], int(df[1].iloc[i])])

df = pd.DataFrame(newrows)
val_dataset = Dataset.from_pandas(pd.DataFrame({'text': df[0], 'emotion': df[1]}))
val_dataset = val_dataset.map(tokenize_function, batched=True)
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'emotion'])
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=8)

Map: 100%|██████████| 4548/4548 [00:01<00:00, 3050.34 examples/s]


In [49]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate(model, dataloader, device):

    model.eval()
    total_correct = 0
    total_labels = []
    total_preds = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['emotion'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted = torch.max(logits, dim=1)

            total_correct += (predicted == labels).sum().item()
            total_labels.extend(labels.cpu().numpy())
            total_preds.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(total_labels, total_preds)
    precision = precision_score(total_labels, total_preds, average='macro')
    recall = recall_score(total_labels, total_preds, average='macro')
    f1 = f1_score(total_labels, total_preds, average='macro')

    return accuracy, precision, recall, f1

In [50]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=28)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(3):
    model.train()
    total_loss = 0
    with tqdm.tqdm(train_dataloader, unit="batch", desc=f"Epoch {epoch+1}") as pbar:
        for batch in pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['emotion'].to(device)

            optimizer.zero_grad()

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            pbar.set_postfix(loss=f"{loss.item():.4f}")
    
    print(f'Epoch {epoch+1}, Average Loss: {total_loss / len(train_dataloader)}')

    model.eval()
    
    accuracy, precision, recall, f1 = evaluate(model, val_dataloader, device)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1:   0%|          | 9/4539 [00:23<3:13:51,  2.57s/batch, loss=3.6982]


KeyboardInterrupt: 